In [1]:
 # Практика

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## Загружаем данные

In [3]:
import pandas as pd

In [4]:
movies = pd.read_csv('movies.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
tags = pd.read_csv('tags.csv')

In [7]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
movies['genres_split'] = movies.genres.str.split('|')

In [9]:
movies['genres_cnt'] = movies.apply(lambda r: len(r['genres_split']), axis=1)

In [10]:
movies.head()

,movieId,title,genres,genres_split,genres_cnt
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",5
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",3
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]",3
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],1


## TFIDF на тэгах

Возьмем только тэги, встечающиеся 5 и более раз

In [60]:
tags_count = tags.groupby(['tag'])['movieId'].count()

In [61]:
tags_filter = tags_count[tags_count>=5].index

In [62]:
tags_filted = tags[tags['tag'].isin(tags_filter)]

In [63]:
tags_filted.shape

(1669, 4)

In [64]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [65]:
tags_vec = CountVectorizer()
tags_matrix = tags_vec.fit_transform(tags_filted['tag'])
tags_vec.vocabulary_

{u'250': 0,
 u'action': 1,
 u'adam': 2,
 u'adolescence': 3,
 u'adultery': 4,
 u'adventure': 5,
 u'africa': 6,
 u'al': 7,
 u'alcoholism': 8,
 u'aliens': 9,
 u'amnesia': 10,
 u'and': 11,
 u'animal': 12,
 u'animation': 13,
 u'anime': 14,
 u'apocalyptic': 15,
 u'appealing': 16,
 u'artificial': 17,
 u'arts': 18,
 u'assassination': 19,
 u'astaire': 20,
 u'atmospheric': 21,
 u'australia': 22,
 u'bad': 23,
 u'baseball': 24,
 u'based': 25,
 u'beautiful': 26,
 u'bible': 27,
 u'biopic': 28,
 u'bittersweet': 29,
 u'black': 30,
 u'book': 31,
 u'boxing': 32,
 u'brad': 33,
 u'brothers': 34,
 u'business': 35,
 u'cerebral': 36,
 u'charles': 37,
 u'christmas': 38,
 u'cinematography': 39,
 u'civil': 40,
 u'classic': 41,
 u'clever': 42,
 u'coen': 43,
 u'comedy': 44,
 u'comic': 45,
 u'commentary': 46,
 u'corruption': 47,
 u'court': 48,
 u'creepy': 49,
 u'crime': 50,
 u'cross': 51,
 u'cult': 52,
 u'dark': 53,
 u'death': 54,
 u'dialogue': 55,
 u'dicaprio': 56,
 u'dickens': 57,
 u'disability': 58,
 u'disney':

In [73]:
tfidf_tags = TfidfTransformer()

tfidf_tags_dense = tfidf_tags.fit_transform(tags_matrix).todense()

In [72]:
columns = [None for i in range(len(tags_vec.vocabulary_))]
for k in tags_vec.vocabulary_:
    columns[tags_vec.vocabulary_[k]] = k

In [74]:
df_tfidf_tags = pd.DataFrame(tfidf_tags_dense, columns=columns)
df_tfidf_tags.shape

(1669, 205)

In [75]:
tags_with_tfidf = pd.concat((tags, df_tfidf_tags), axis=1)
tags_with_tfidf.head()

,userId,movieId,tag,timestamp,250,action,adam,adolescence,adultery,adventure,...,violence,visually,war,wedding,white,will,witty,world,york,zombies
0,2,60756,funny,1445714994,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,60756,Highly quotable,1445714996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,60756,will ferrell,1445714992,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,89774,Boxing story,1445715207,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,89774,MMA,1445715200,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
movie_tags = tags_with_tfidf.drop(columns=['userId', 'timestamp']).groupby('movieId').sum()

## TFIDF на жанрах

In [80]:
movies['genres_space'] = movies.apply(lambda r: ' '.join(r['genres_split']), axis=1)

In [81]:
movies.head()

,movieId,title,genres,genres_split,genres_cnt,genres_space
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",5,Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",3,Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",2,Comedy Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]",3,Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],1,Comedy


In [82]:
cnt_vec = CountVectorizer()

In [83]:
processed = cnt_vec.fit_transform(movies['genres_space'])

In [84]:
tfidf = TfidfTransformer()

In [85]:
tfidf_dense = tfidf.fit_transform(processed).todense()

In [86]:
tfidf_dense.shape

(9742L, 24L)

In [87]:
cnt_vec.vocabulary_

{u'action': 0,
 u'adventure': 1,
 u'animation': 2,
 u'children': 3,
 u'comedy': 4,
 u'crime': 5,
 u'documentary': 6,
 u'drama': 7,
 u'fantasy': 8,
 u'fi': 9,
 u'film': 10,
 u'genres': 11,
 u'horror': 12,
 u'imax': 13,
 u'listed': 14,
 u'musical': 15,
 u'mystery': 16,
 u'no': 17,
 u'noir': 18,
 u'romance': 19,
 u'sci': 20,
 u'thriller': 21,
 u'war': 22,
 u'western': 23}

In [88]:
columns = [None for i in range(len(cnt_vec.vocabulary_))]
for k in cnt_vec.vocabulary_:
    columns[cnt_vec.vocabulary_[k]] = k

In [89]:
df_tfidf = pd.DataFrame(tfidf_dense, columns=columns)

In [90]:
df_tfidf.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [91]:
movies_with_tfidf = pd.concat((movies, df_tfidf), axis=1)

In [92]:
movies_with_tfidf.head()

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",5,Adventure Animation Children Comedy Fantasy,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",3,Adventure Children Fantasy,0.0,0.512361,0.000000,0.620525,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",2,Comedy Romance,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]",3,Comedy Drama Romance,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],1,Comedy,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [93]:
import re

In [94]:
re.findall(r'\(([0-9]{4})\)', 'Toy Story (1995)')[0]

'1995'

In [95]:
movies_with_tfidf['year'] = movies_with_tfidf.apply(lambda r: int(re.findall(r'\(([0-9]{4})\)', r['title'])[0]) if len(re.findall(r'\(([0-9]{4})\)', r['title'])) else None, axis=1)

In [96]:
movies_with_tfidf['year'] = movies_with_tfidf['year'].fillna(2002)

In [97]:
movies_with_tfidf.columns

Index([     u'movieId',        u'title',       u'genres', u'genres_split',
         u'genres_cnt', u'genres_space',       u'action',    u'adventure',
          u'animation',     u'children',       u'comedy',        u'crime',
        u'documentary',        u'drama',      u'fantasy',           u'fi',
               u'film',       u'genres',       u'horror',         u'imax',
             u'listed',      u'musical',      u'mystery',           u'no',
               u'noir',      u'romance',          u'sci',     u'thriller',
                u'war',      u'western',         u'year'],
      dtype='object')

In [98]:
fc = ['movieId', 'title', 'genres_cnt', 'action', 'adventure', 'animation', 'children',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi', 'film',
        'horror', 'imax', 'listed', 'musical', 'mystery', 'no',
       'noir', 'romance', 'sci', 'thriller', 'war', 'western', 'year']

In [99]:
movies_for_ds = movies_with_tfidf[fc]

In [103]:
movies_for_ds = movies_for_ds.merge(movie_tags, how='left', on='movieId').fillna(0)

In [130]:
movies_for_ds

,movieId,title,genres_cnt,action_x,adventure_x,animation_x,children,comedy_x,crime_x,documentary,...,violence,visually,war_y,wedding,white,will,witty,world,york,zombies
0,1,Toy Story (1995),5,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),3,0.000000,0.512361,0.000000,0.620525,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),2,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),3,0.000000,0.000000,0.000000,0.000000,0.505015,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),1,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),3,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),2,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),2,0.000000,0.636699,0.000000,0.771112,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),3,0.553065,0.629522,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


## CBRS For User

In [111]:
ratings = pd.read_csv('ratings.csv')

In [112]:
movies_with_ratings = pd.merge(ratings, movies_for_ds, on='movieId')

In [113]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,genres_cnt,action_x,adventure_x,animation_x,children,...,violence,visually,war_y,wedding,white,will,witty,world,york,zombies
0,1,1,4.0,964982703,Toy Story (1995),5,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,847434962,Toy Story (1995),5,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,1106635946,Toy Story (1995),5,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,2.5,1510577970,Toy Story (1995),5,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,4.5,1305696483,Toy Story (1995),5,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [117]:
import numpy as np

In [118]:
TARGET_USER = 474

In [119]:
df_for_user = movies_with_ratings[movies_with_ratings['userId']==TARGET_USER]

In [121]:
X, y = df_for_user.drop(columns=['userId', 'movieId', 'rating', 'timestamp', 'title']), df_for_user['rating']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [123]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [124]:
models = [LinearRegression, Lasso, Ridge, RandomForestRegressor, SVR]

In [127]:
for m in models:
    model = m()
    model.fit(X_train, y_train)
    print("{}. r2_train: {:.4f}, r2_test: {:.4f}, mae_train: {:.4f}, mae_test: {:.4f}, RMSE_test: {:.4f}".format(
        m.__name__, model.score(X_train ,y_train), model.score(X_test, y_test),
        mean_absolute_error(model.predict(X_train), y_train),
        mean_absolute_error(model.predict(X_test), y_test),
        np.sqrt(mean_squared_error(model.predict(X_test), y_test))
    ))

LinearRegression. r2_train: 0.2230, r2_test: -60030841022614382667890688.0000, mae_train: 0.5631, mae_test: 314101201969.9555, RMSE_test: 6452467470909.6504
Lasso. r2_train: 0.0000, r2_test: -0.0069, mae_train: 0.6316, mae_test: 0.6321, RMSE_test: 0.8357
Ridge. r2_train: 0.2423, r2_test: -0.1194, mae_train: 0.5524, mae_test: 0.6626, RMSE_test: 0.8811
RandomForestRegressor. r2_train: 0.7234, r2_test: 0.0946, mae_train: 0.3124, mae_test: 0.6184, RMSE_test: 0.7924
SVR. r2_train: 0.2913, r2_test: 0.0345, mae_train: 0.4929, mae_test: 0.6035, RMSE_test: 0.8183


### RandomForester показал лучший результат

## Item 2 Item

In [128]:
from sklearn.neighbors import NearestNeighbors

In [132]:
X_unsup= movies_for_ds.drop(columns=['movieId', 'title', 'genres_cnt'])b

In [137]:
cl = X_unsup.columns

In [133]:
nn = NearestNeighbors(n_neighbors=10, metric='minkowski', p=2)

In [134]:
nn.fit(X_unsup)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [135]:
movies_for_ds[movies_for_ds['movieId'] == 188751]

,movieId,title,genres_cnt,action_x,adventure_x,animation_x,children,comedy_x,crime_x,documentary,...,violence,visually,war_y,wedding,white,will,witty,world,york,zombies
9715,188751,Mamma Mia: Here We Go Again! (2018),2,0.0,0.0,0.0,0.0,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
nn.kneighbors(movies_for_ds[movies_for_ds['movieId'] == 188751][cl])

(array([[0.        , 0.92637422, 0.92637422, 0.92637422, 0.92637422,
         0.92637422, 0.92637422, 1.        , 1.0775782 , 1.0775782 ]]),
 array([[9715, 9704, 9716, 9685, 9684, 9695, 9686, 9466, 9718, 9698]],
       dtype=int64))

In [139]:
neigbors = nn.kneighbors(movies_for_ds[movies_for_ds['movieId'] == 188751][cl])[1]

In [140]:
neigbors.reshape(-1)

array([9715, 9704, 9716, 9685, 9684, 9695, 9686, 9466, 9718, 9698],
      dtype=int64)

In [141]:
movies_for_ds.iloc[neigbors.reshape(-1)]

,movieId,title,genres_cnt,action_x,adventure_x,animation_x,children,comedy_x,crime_x,documentary,...,violence,visually,war_y,wedding,white,will,witty,world,york,zombies
9715,188751,Mamma Mia: Here We Go Again! (2018),2,0.0,0.0,0.0,0.0,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9704,185473,Blockers (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9716,188797,Tag (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9685,183959,Tom Segura: Disgraceful (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9684,183911,The Clapper (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9695,184791,Fred Armisen: Standup for Drummers (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9686,184015,When We First Met (2018),1,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9466,168326,The Big Sick (2017),2,0.0,0.0,0.0,0.0,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9718,189043,Boundaries (2018),2,0.0,0.0,0.0,0.0,0.734632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9698,184997,"Love, Simon (2018)",2,0.0,0.0,0.0,0.0,0.734632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
